<a href="https://colab.research.google.com/github/ckdlsqpf/projectmanager/blob/main/KLUE_BERT_%EB%AC%B8%EC%84%9C%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate evaluate

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")


from transformers import AutoModelForSequenceClassification
seq_model = AutoModelForSequenceClassification.from_pretrained(
    "klue/bert-base", num_labels=7,
    id2label={
        0: 'IT과학', 1: '경제',  2: '사회',  3: '생활문화', 4: '세계',
        5: '스포츠',  6: '정치'})
seq_model.to('cuda:0')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [3]:
from datasets import load_dataset
ynat_train = load_dataset('klue', 'ynat', split='train')
ynat_val = load_dataset('klue', 'ynat', split='validation')

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [7]:
ynat_train[4]

{'guid': 'ynat-v1_train_00004',
 'title': '회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=243&oid=001&aid=0009999529',
 'date': '2018.04.03. 오전 7:05'}

In [8]:
def convert_example(example):
    result = tokenizer(example['title'])
    result['label'] = [example['label']]
    return result

ynat_train_dataset = ynat_train.map(convert_example)
ynat_val_dataset = ynat_val.map(convert_example)

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [10]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,  # 1번 돌려라
    evaluation_strategy="epoch")

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # 길이 맞춰줘라

trainer = Trainer(
    model=seq_model,
    args=training_args,
    train_dataset=ynat_train_dataset,
    eval_dataset=ynat_val_dataset,
    data_collator=data_collator
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.347400,0.408696


TrainOutput(global_step=5710, training_loss=0.452312224967676, metrics={'train_runtime': 601.2872, 'train_samples_per_second': 75.967, 'train_steps_per_second': 9.496, 'total_flos': 463740691731900.0, 'train_loss': 0.452312224967676, 'epoch': 1.0})

In [12]:
from transformers import pipeline
text_cls = pipeline('text-classification', model=seq_model,
              tokenizer=tokenizer, device='cuda:0')

text_cls('탕후루 인기가 치솟아')

[{'label': '생활문화', 'score': 0.6793088316917419}]